In [228]:
from turtle import mode
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("original_datasets/emdat_disasters.csv", encoding="utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15784 entries, 0 to 15783
Data columns (total 46 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   DisNo.                                     15784 non-null  object 
 1   Historic                                   15784 non-null  object 
 2   Classification Key                         15784 non-null  object 
 3   Disaster Group                             15784 non-null  object 
 4   Disaster Subgroup                          15784 non-null  object 
 5   Disaster Type                              15784 non-null  object 
 6   Disaster Subtype                           15784 non-null  object 
 7   External IDs                               2405 non-null   object 
 8   Event Name                                 4955 non-null   object 
 9   ISO                                        15784 non-null  object 
 10  Country               

In [229]:
def removeColumns(df):
    return df.drop(
        columns=[
            "Historic",
            "Classification Key",
            "External IDs",
            "Entry Date",
            "Last Update",
            "Admin Units",
            "Disaster Subtype",
            "Origin",
            "River Basin",
            "Location",
            "End Year",
            "End Month",
            "End Day",
            "OFDA/BHA Response",
            "Appeal",
            "Declaration",
            "CPI",
            "Reconstruction Costs ('000 US$)",
            "Reconstruction Costs, Adjusted ('000 US$)",
            "Associated Types",
            "AID Contribution ('000 US$)",
            "Magnitude",
            "Magnitude Scale",
            "Latitude",
            "Longitude",
            "Start Month",
            "Start Day",
            "No. Injured",
            "No. Affected",
            "No. Homeless",
            "Total Affected",
            "Insured Damage ('000 US$)",
            "Insured Damage, Adjusted ('000 US$)",
            "Total Damage ('000 US$)",
            "Event Name",
            "Total Damage, Adjusted ('000 US$)",
        ],
        axis=1,
    )
df = df.rename(columns={"DisNo.": "DisNo", "Disaster Group": "DisasterGroup", "Disaster Subgroup": "DisasterSubgroup", "Disaster Type": "DisasterType", "Start Year": "StartYear", "Total Deaths": "TotalDeaths"})
df = df.pipe(removeColumns)

df['TotalDeaths'] = df['TotalDeaths'].fillna(0)

emdat_disasters = df
emdat_disasters

,DisNo,DisasterGroup,DisasterSubgroup,DisasterType,ISO,Country,Subregion,Region,StartYear,TotalDeaths
0,1999-9388-DJI,Natural,Climatological,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,2001,0.0
1,1999-9388-SDN,Natural,Climatological,Drought,SDN,Sudan,Northern Africa,Africa,2000,0.0
2,1999-9388-SOM,Natural,Climatological,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,2000,21.0
3,2000-0001-AGO,Technological,Transport,Road,AGO,Angola,Sub-Saharan Africa,Africa,2000,14.0
4,2000-0002-AGO,Natural,Hydrological,Flood,AGO,Angola,Sub-Saharan Africa,Africa,2000,31.0
...,...,...,...,...,...,...,...,...,...,...
15779,2024-9075-COL,Natural,Climatological,Drought,COL,Colombia,Latin America and the Caribbean,Americas,2024,0.0
15780,2024-9131-ZMB,Natural,Climatological,Drought,ZMB,Zambia,Sub-Saharan Africa,Africa,2024,0.0
15781,2024-9135-MHL,Natural,Climatological,Drought,MHL,Marshall Islands,Micronesia,Oceania,2024,0.0
15782,2024-9197-MWI,Natural,Climatological,Drought,MWI,Malawi,Sub-Saharan Africa,Africa,2024,0.0


In [230]:
country_data = pd.read_csv("original_datasets/world-data-2023.csv", encoding="utf-8")

# Correct the names of countries in both datasets that don't match
df.loc[df['Country']=='Türkiye', 'Country'] = 'Turkey'
df.loc[df['Country']=='United States of America', 'Country'] = 'United States'
df.loc[df['Country']=='Russian Federation', 'Country'] = 'Russia'
df.loc[df['Country']=='Czechia', 'Country'] = 'Czech Republic'
df.loc[df['Country']=='Netherlands (Kingdom of the)', 'Country'] = 'Netherlands'
df.loc[df['Country']=="Democratic People's Republic of Korea", 'Country'] = 'North Korea'
df.loc[df['Country']=="Republic of Korea", 'Country'] = 'South Korea'
df.loc[df['Country']=="Congo", 'Country'] = 'Republic of the Congo'
df.loc[df['Country']=="Iran (Islamic Republic of)", 'Country'] = 'Iran'
df.loc[df['Country']=="United Republic of Tanzania", 'Country'] = 'Tanzania'
df.loc[df['Country']=="Viet Nam", 'Country'] = 'Vietnam'
df.loc[df['Country']=="Syrian Arab Republic", 'Country'] = 'Syria'
df.loc[df['Country']=="Cabo Verde", 'Country'] = 'Cape Verde'
df.loc[df['Country']=="Venezuela (Bolivarian Republic of)", 'Country'] = 'Venezuela'
df.loc[df['Country']=="Timor-Leste", 'Country'] = 'East Timor'
df.loc[df['Country']=="Bolivia (Plurinational State of)", 'Country'] = 'Bolivia'
df.loc[df['Country']=="Republic of Moldova", 'Country'] = 'Moldova'
df.loc[df['Country']=="Ireland", 'Country'] = 'Republic of Ireland'
df.loc[df['Country']=="United Kingdom of Great Britain and Northern Ireland", 'Country'] = 'United Kingdom'
df.loc[df['Country']=="Micronesia (Federated States of)", 'Country'] = 'Federated States of Micronesia'
df.loc[df['Country']=="State of Palestine", 'Country'] = 'Palestinian National Authority'
df.loc[df['Country']=="Bahamas", 'Country'] = 'The Bahamas'

# Discard countries with no match on the two datasets (doing so only discards about 362 events which is a small percentage of the dataset (~2.3%))
non_overlapping_countries = [country for country in country_data['Country'].to_list() if country not in list(df['Country'].unique())]
for country in non_overlapping_countries:
    country_data = country_data.drop(country_data[(country_data['Country'] == country)].index)

non_overlapping_countries = [country for country in list(df['Country'].unique()) if country not in country_data['Country'].to_list()]
for country in non_overlapping_countries:
    df = df.drop(df[(df['Country'] == country)].index)

# keep only desired columns
columns_to_keep = ['Country', 'Land Area(Km2)']
country_data = country_data[columns_to_keep]
country_data['Land Area(Km2)'] = country_data['Land Area(Km2)'].str.replace(',', '').astype('Int64')
country_data.rename(columns={"Land Area(Km2)": "land_area"}, inplace=True)

# Remove palestine from the dataset
country_data = country_data[country_data['Country'] != 'Palestinian National Authority']

country_data

,Country,land_area
0,Afghanistan,652230
1,Albania,28748
2,Algeria,2381741
4,Angola,1246700
5,Antigua and Barbuda,443
...,...,...
190,Venezuela,912050
191,Vietnam,331210
192,Yemen,527968
193,Zambia,752618


In [231]:
cum_disasters = []
cum_deaths = []
#for country in list(country_data['Country']):
#    cum_disasters += [df['Country'].value_counts()[country]]
#    cum_deaths += [sum(df.loc[df['Country']==country, 'TotalDeaths'])]

#country_data['Total number of disasters'] = cum_disasters
#country_data['Total number of Deaths'] = cum_deaths
#country_data['Num of disasters/area'] = country_data['Total number of disasters']/country_data['Land Area(Km2)']

country_data

,Country,land_area
0,Afghanistan,652230
1,Albania,28748
2,Algeria,2381741
4,Angola,1246700
5,Antigua and Barbuda,443
...,...,...
190,Venezuela,912050
191,Vietnam,331210
192,Yemen,527968
193,Zambia,752618


In [232]:
df

,DisNo,DisasterGroup,DisasterSubgroup,DisasterType,ISO,Country,Subregion,Region,StartYear,TotalDeaths
0,1999-9388-DJI,Natural,Climatological,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,2001,0.0
1,1999-9388-SDN,Natural,Climatological,Drought,SDN,Sudan,Northern Africa,Africa,2000,0.0
2,1999-9388-SOM,Natural,Climatological,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,2000,21.0
3,2000-0001-AGO,Technological,Transport,Road,AGO,Angola,Sub-Saharan Africa,Africa,2000,14.0
4,2000-0002-AGO,Natural,Hydrological,Flood,AGO,Angola,Sub-Saharan Africa,Africa,2000,31.0
...,...,...,...,...,...,...,...,...,...,...
15779,2024-9075-COL,Natural,Climatological,Drought,COL,Colombia,Latin America and the Caribbean,Americas,2024,0.0
15780,2024-9131-ZMB,Natural,Climatological,Drought,ZMB,Zambia,Sub-Saharan Africa,Africa,2024,0.0
15781,2024-9135-MHL,Natural,Climatological,Drought,MHL,Marshall Islands,Micronesia,Oceania,2024,0.0
15782,2024-9197-MWI,Natural,Climatological,Drought,MWI,Malawi,Sub-Saharan Africa,Africa,2024,0.0


In [233]:
# save the DataFrames to a new CSV files
df.to_csv('emdat_disasters_cleaned.csv', index=False, mode='w', encoding='utf-8')
country_data.to_csv('country_data_processed.csv', index=False, mode='w', encoding='utf-8')


In [234]:
# create a new DataFrame with the number of disasters and deaths per year per contry, region and subregion and save it to a new CSV file
# Group by Year, Country, Region, and Subregion and calculate the number of disasters and total deaths
disasters_per_country = df.groupby(['StartYear', 'Country', 'ISO']).agg(
    num_disasters=('DisNo', 'count'),
    total_deaths=('TotalDeaths', 'sum')
).reset_index()

# Merge the new DataFrame with the country_data DataFrame to add the Land Area(Km2) column
disasters_per_country = pd.merge(disasters_per_country, country_data, on='Country')

#Calculating the number of disasters per area
disasters_per_country['disasters_per_area'] = (disasters_per_country['num_disasters'] / disasters_per_country['land_area']) * 10**6


# Save the new DataFrame to a CSV file
disasters_per_country.to_csv(
    "pre-processing/disasters_per_country.csv", index=False, mode="w", encoding="utf-8"
)

disasters_per_country

,StartYear,Country,ISO,num_disasters,total_deaths,land_area,disasters_per_area
0,2000,Afghanistan,AFG,6,634.0,652230,9.199209
1,2000,Algeria,DZA,6,70.0,2381741,2.519166
2,2000,Angola,AGO,14,294.0,1246700,11.229646
3,2000,Argentina,ARG,9,48.0,2780400,3.236944
4,2000,Armenia,ARM,1,0.0,29743,33.621356
...,...,...,...,...,...,...,...
3163,2024,United States,USA,7,97.0,9833517,0.711851
3164,2024,Uruguay,URY,1,0.0,176215,5.674886
3165,2024,Vietnam,VNM,1,1.0,331210,3.019233
3166,2024,Zambia,ZMB,1,0.0,752618,1.328695


In [235]:
# Group by Year and Subregion and calculate the number of disasters and total deaths
disasters_per_subregion = df.groupby(['StartYear', 'Subregion']).agg(
    num_disasters=('DisNo', 'count'),
    total_deaths=('TotalDeaths', 'sum')
).reset_index()

# Save the new DataFrame to a CSV file
disasters_per_subregion.to_csv('pre-processing/disasters_per_subregion.csv', index=False, mode='w', encoding='utf-8')

disasters_per_subregion

,StartYear,Subregion,num_disasters,total_deaths
0,2000,Australia and New Zealand,10,182.0
1,2000,Central Asia,7,22.0
2,2000,Eastern Asia,99,2784.0
3,2000,Eastern Europe,59,719.0
4,2000,Latin America and the Caribbean,108,1738.0
...,...,...,...,...
400,2024,Southern Asia,21,1959.0
401,2024,Southern Europe,4,12.0
402,2024,Sub-Saharan Africa,40,1253.0
403,2024,Western Asia,10,73.0


In [236]:
# Group by Year and Region and calculate the number of disasters and total deaths
disasters_per_region = df.groupby(['StartYear', 'Region']).agg(
    num_disasters=('DisNo', 'count'),
    total_deaths=('TotalDeaths', 'sum')
).reset_index()

# Save the new DataFrame to a CSV file
disasters_per_region.to_csv(
    "pre-processing/disasters_per_region.csv", index=False, mode="w", encoding="utf-8"
)

disasters_per_region

,StartYear,Region,num_disasters,total_deaths
0,2000,Africa,253,10345.0
1,2000,Americas,149,2054.0
2,2000,Asia,335,13313.0
3,2000,Europe,120,1510.0
4,2000,Oceania,15,231.0
...,...,...,...,...
120,2024,Africa,42,1283.0
121,2024,Americas,41,1029.0
122,2024,Asia,63,2772.0
123,2024,Europe,9,43.0
